DATASET LINK : https://www.kaggle.com/mkechinov/ecommerce-behavior-data-from-multi-category-store?fbclid=IwAR3Rji50q1WtaHD8By7X-_E6WO52Ksj7SQmulpV7-flbmhQKerdWR1M2cdc&select=2019-Oct.csv

In [1]:
%autosave 10
import time
import os
import pandas as pd

Autosaving every 10 seconds


In [2]:
#Size of the file in bytes
os.path.getsize('2019-Oct.csv')

5668612855

In [4]:
from dask import dataframe as dd
start = time.time()
dask_df = dd.read_csv("2019-Oct.csv")
end = time.time()
print("Reading csv with dask takes: ",(end-start),"sec")

Reading csv with dask takes:  0.0352177619934082 sec


In [5]:
#import using dask
from dask import dataframe as dd
df = dd.read_csv('2019-Oct.csv',delimiter=',')

In [9]:
#no of rows
len(df.index)

42448764

In [10]:
#no of columns
len(df.columns)

9

In [11]:
# remove special character
df.columns=df.columns.str.replace('[#,@,&]','')

In [12]:
#To remove white space from columns
df.columns = df.columns.str.replace(' ', '')

In [13]:
data=df.columns
data

Index(['event_time', 'event_type', 'product_id', 'category_id',
       'category_code', 'brand', 'price', 'user_id', 'user_session'],
      dtype='object')

In [14]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
   
    
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Writing testutility.py


In [15]:
%%writefile file.yaml
file_type: csv
dataset_name: 2019-Oct
file_name: 2019-Oct
table_name: 2019-Oct
inbound_delimiter: ","
outbound_delimiter: ","
skip_leading_rows: 1
columns:
    - event_time 
    - event_type 
    - product_id 
    - category_id
    - category_code 
    - brand 
    - price 
    - user_id
    - user_session

Writing file.yaml


In [16]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")

In [17]:
#inspecting data of config file
config_data

{'file_type': 'csv',
 'dataset_name': '2019-Oct',
 'file_name': '2019-Oct',
 'table_name': '2019-Oct',
 'inbound_delimiter': ',',
 'outbound_delimiter': ',',
 'skip_leading_rows': 1,
 'columns': ['event_time',
  'event_type',
  'product_id',
  'category_id',
  'category_code',
  'brand',
  'price',
  'user_id',
  'user_session']}

In [18]:
import pandas as pd

In [19]:
file_type= config_data['file_type']
source_file= "./" + config_data['file_name'] + f'.{file_type}'

df= pd.read_csv(source_file, config_data['inbound_delimiter'])

In [20]:
util.col_header_val(df, config_data)

column name and column length validation passed


1

In [21]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['event_time', 'event_type', 'product_id', 'category_id',
       'category_code', 'brand', 'price', 'user_id', 'user_session'],
      dtype='object')
columns of YAML are: ['event_time', 'event_type', 'product_id', 'category_id', 'category_code', 'brand', 'price', 'user_id', 'user_session']


In [22]:
if util.col_header_val(df, config_data)==0:
    print("validation is failed")
    
else:
    print("validation is passed")

column name and column length validation passed
validation is passed


In [24]:
import datetime
import csv
import gzip

from dask import dataframe as dd
df = dd.read_csv("2019-Oct.csv",delimiter=',')

# Write csv in gz format in pipe separated text file (|)
df.to_csv('2019-Oct.csv.gz',
          sep='|',
          header=True,
          index=False,
          quoting=csv.QUOTE_ALL,
          compression='gzip',
          quotechar='"',
          doublequote=True,
          line_terminator='\n')

['C:/Users/Shiwani/Desktop/Week 6/2019-Oct.csv.gz\\00.part',
 'C:/Users/Shiwani/Desktop/Week 6/2019-Oct.csv.gz\\01.part',
 'C:/Users/Shiwani/Desktop/Week 6/2019-Oct.csv.gz\\02.part',
 'C:/Users/Shiwani/Desktop/Week 6/2019-Oct.csv.gz\\03.part',
 'C:/Users/Shiwani/Desktop/Week 6/2019-Oct.csv.gz\\04.part',
 'C:/Users/Shiwani/Desktop/Week 6/2019-Oct.csv.gz\\05.part',
 'C:/Users/Shiwani/Desktop/Week 6/2019-Oct.csv.gz\\06.part',
 'C:/Users/Shiwani/Desktop/Week 6/2019-Oct.csv.gz\\07.part',
 'C:/Users/Shiwani/Desktop/Week 6/2019-Oct.csv.gz\\08.part',
 'C:/Users/Shiwani/Desktop/Week 6/2019-Oct.csv.gz\\09.part',
 'C:/Users/Shiwani/Desktop/Week 6/2019-Oct.csv.gz\\10.part',
 'C:/Users/Shiwani/Desktop/Week 6/2019-Oct.csv.gz\\11.part',
 'C:/Users/Shiwani/Desktop/Week 6/2019-Oct.csv.gz\\12.part',
 'C:/Users/Shiwani/Desktop/Week 6/2019-Oct.csv.gz\\13.part',
 'C:/Users/Shiwani/Desktop/Week 6/2019-Oct.csv.gz\\14.part',
 'C:/Users/Shiwani/Desktop/Week 6/2019-Oct.csv.gz\\15.part',
 'C:/Users/Shiwani/Deskt

In [25]:
#size of the gz format folder in bytes
os.path.getsize('2019-Oct.csv.gz')

32768

In [26]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 9 entries, event_time to user_session
dtypes: object(5), float64(1), int64(3)

In [27]:
data=df.columns
data

Index(['event_time', 'event_type', 'product_id', 'category_id',
       'category_code', 'brand', 'price', 'user_id', 'user_session'],
      dtype='object')

In [28]:
#no of Columns
len(df.columns)

9

In [29]:
#no of Rows
len(df.index)

42448764